# light GBM

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [5]:
from warnings import filterwarnings
filterwarnings('ignore')

In [6]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League','Division','NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
X = pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,
                                                y,
                                                test_size=0.25,
                                                random_state=42)

### model & tahmin

In [8]:
from lightgbm import LGBMRegressor

In [9]:
lgb_model=LGBMRegressor().fit(X_train, y_train)

In [10]:
y_pred = lgb_model.predict(X_test)

In [11]:
np.sqrt(mean_squared_error(y_test,y_pred))

363.8712087611089

In [22]:
lgbm_params ={"learning_rate":[0.01,0.1,0.5,1],
              "n_estimators":[20,40,100,200,1000],
             "max_depth":[0.5,1,2,3,6]}

In [23]:
lgbm_cv_model = GridSearchCV(lgb_model, lgbm_params,
                             cv=10,
                             n_jobs=-1,
                             verbose=2).fit(X_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 877 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    6.3s finished


In [24]:
lgbm_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20}

In [25]:
lgb_model=LGBMRegressor(learning_rate=0.1,max_depth=6,n_estimators=20).fit(X_train, y_train)

In [26]:
y_pred=lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

371.5044868943621